In [ ]:
!mkdir ~p ~/.kaggle
!cp kaggle.json ~/.kaggle/


cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
100% 1.06G/1.06G [00:27<00:00, 40.6MB/s]
100% 1.06G/1.06G [00:27<00:00, 41.9MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [ ]:
train_transforms=transforms.Compose([
    transforms.Resize(128),transforms.RandomResizedCrop(128),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225] )])
val_transforms=transforms.Compose([transforms.Resize(128),transforms.CenterCrop(128),transforms.ToTensor(),
                                   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])


In [ ]:
train_dataset=datasets.ImageFolder('/content/train',transform=train_transforms)
val_dataset=datasets.ImageFolder('/content/test',transform=val_transforms)
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
val_loader=DataLoader(val_dataset,batch_size=32,shuffle=True)

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.convolayer=nn.Sequential(
        nn.Conv2d(3,32,kernel_size=3,padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2,2),


        nn.Conv2d(32,64,kernel_size=3,padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2,2),

        nn.Conv2d(64,128,kernel_size=3,padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2,2),

    )
    self.fc=nn.Sequential(
        nn.Dropout(p=0.5),
        nn.Linear(128*16*16,500),
        nn.ReLU(inplace=True),
        nn.Linear(500,2)
    )
  def forward(self,x):
     x=self.convolayer(x)
     x=x.view(x.size(0),-1)
     x=self.fc(x)
     return x

In [ ]:
#initialize the model
model=CNN()
#loss function
criterion=nn.CrossEntropyLoss()
#optimizer
optimizer=optim.Adam(model.parameters(),lr=0.001)

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

CNN(
  (convolayer): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=32768, out_features=500, bias=True)
    (2): 

In [ ]:
def train_model(model,criterion,optimizer,train_loader,val_loader,epochs=10):
  train_acc_list=[]
  val_acc_list=[]

  for epoch in range(epochs):
    model.train()
    running_loss=0.0
    total=0.0
    correct=0;
    for batch_idx, (inputs, labels) in enumerate(train_loader):
      inputs, labels = inputs.to(device), labels.to(device)
      optimizer.zero_grad()
      outputs=model(inputs)
      loss=criterion(outputs,labels)
      loss.backward()
      optimizer.step()
      running_loss+=loss.item()
      _,predicted=torch.max(outputs.data,1)
      total+=labels.size(0)
      correct+=(predicted==labels).sum().item()
      if batch_idx % 10 == 0:
                print(f"Epoch {epoch+1}/{epochs}, Batch {batch_idx}/{len(train_loader)} - Loss: {running_loss/10:.4f}")
                running_loss = 0.0

  train_acc=100*correct/total
  train_acc_list.append(train_acc)
  model.eval()
  correct = 0
  total = 0

  with torch.no_grad():
   for inputs, labels in val_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

   val_acc = correct / total
   val_acc_list.append(val_acc)

   print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

  return train_acc_list, val_acc_list

# Train the model for 10 epochs
train_acc, val_acc = train_model(model, criterion, optimizer, train_loader, val_loader,epochs=10)

# Plot training and validation accuracy
plt.plot(train_acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()



Epoch 1/10, Batch 0/625 - Loss: 0.0707
Epoch 1/10, Batch 10/625 - Loss: 6.1547
Epoch 1/10, Batch 20/625 - Loss: 1.2841
Epoch 1/10, Batch 30/625 - Loss: 1.0589
Epoch 1/10, Batch 40/625 - Loss: 0.7720
Epoch 1/10, Batch 50/625 - Loss: 0.7274
Epoch 1/10, Batch 60/625 - Loss: 0.7902
Epoch 1/10, Batch 70/625 - Loss: 0.9479
Epoch 1/10, Batch 80/625 - Loss: 0.9448
Epoch 1/10, Batch 90/625 - Loss: 0.7899
Epoch 1/10, Batch 100/625 - Loss: 0.7524
Epoch 1/10, Batch 110/625 - Loss: 0.7031
Epoch 1/10, Batch 120/625 - Loss: 0.6799
Epoch 1/10, Batch 130/625 - Loss: 0.6524
Epoch 1/10, Batch 140/625 - Loss: 0.6414
Epoch 1/10, Batch 150/625 - Loss: 0.6681
Epoch 1/10, Batch 160/625 - Loss: 0.6494
Epoch 1/10, Batch 170/625 - Loss: 0.6614
Epoch 1/10, Batch 180/625 - Loss: 0.6553
Epoch 1/10, Batch 190/625 - Loss: 0.6779
Epoch 1/10, Batch 200/625 - Loss: 0.6271
Epoch 1/10, Batch 210/625 - Loss: 0.6570
Epoch 1/10, Batch 220/625 - Loss: 0.6566
Epoch 1/10, Batch 230/625 - Loss: 0.6400
Epoch 1/10, Batch 240/625 -

KeyboardInterrupt: 

NameError: name 'train_acc' is not defined

# New Section

# New Section